# Extract Structured Data from Text

In this notebook we'll go through a structured data extraction example, using the Gemini API to extract the lists of characters, relationships, things, and places from a story.

The code in this notebook is adapted from Google's [Gemini tutorial](https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/tutorials/extract_structured_data.ipynb).


## Setup

Before we begin, please make sure you have setup the `.env` file in the project 
directory as described in [`README.md`](README.md).

Next, we will use 
[`python-dotenv`](https://github.com/theskumar/python-dotenv)
to load in the necessary environment variables for this notebook:

In [1]:
import os
from dotenv import load_dotenv

_ = load_dotenv()

assert os.environ.get("GOOGLE_API_KEY")

You can pass the API key to the Google AI Python SDK in two ways:
* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`


In [2]:
import google.generativeai as genai

genai.configure(
    api_key=os.environ.get(
        "GOOGLE_API_KEY"
    )
)

## The example task

In this notebook we will extract entities from a story written by **Gemini 1.5 Pro**:

In [3]:
from IPython.display import Markdown

story = """In the quaint town of Willow Creek, nestled amidst rolling hills and whispering willows, resided a young girl named Anya. As she stepped out of the creaky wooden door of her modest cottage, her heart skipped a beat with excitement and anticipation. Today was her first day of school, and she couldn't wait to show off her prized possession - a magical backpack.\n\nHanded down to her from her grandmother, the backpack was no ordinary satchel. Its soft, emerald-green fabric shimmered with an ethereal glow, and its leather straps held secrets that only Anya knew. Within its capacious interior lay an enchanted world, filled with wonders that would ignite her imagination and change her life forever.\n\nAnya's parents, kind-hearted Elise and wise-bearded Edward, bid her farewell with warm embraces. "Remember, my dear," whispered her mother, "use your magic wisely and for good." Her father added, "Always seek knowledge, and let the backpack be your trusted companion."\n\nWith a skip in her step, Anya set off towards the town's only schoolhouse. On her way, she passed her best friend, Samuel, a curious and adventurous boy with a mischievous grin. "Hey, Anya," he called out. "Can I see your backpack?"\n\nAnya hesitated for a moment before unzipping the flap and revealing its contents. Samuel's eyes widened in amazement as he peered inside. There, nestled amidst pencils and notebooks, were a shimmering sword, a book of ancient spells, a tiny compass that always pointed north, and a magical key that could open any lock.\n\nTogether, they marveled at the backpack's wonders, promising to keep its secrets safe. As they approached the schoolhouse, Anya noticed a group of older children huddled together, their faces etched with fear. Curiosity getting the better of her, she cautiously approached.\n\n"What's wrong?" she asked.\n\nA tall, lanky boy stepped forward. "There's a monster in the forest," he stammered. "It's been terrorizing the town, attacking animals and even people."\n\nAnya's heart sank. The town of Willow Creek was small and peaceful, and the thought of a monster brought a shiver down her spine. She knew she had to do something to protect her family and friends.\n\nWithout a moment's hesitation, Anya opened her backpack and retrieved the shimmering sword. With a determined gleam in her eye, she turned to her terrified peers. "Don't worry," she said, her voice steady. "I'll take care of it."\n\nWith Samuel close behind her, Anya ventured into the shadowy depths of the forest. The trees seemed to whisper secrets as she passed, and the undergrowth rustled with unseen creatures. As they walked deeper into the forest, the air grew heavy and the ground beneath their feet trembled.\n\nSuddenly, they came to a clearing, and there before their eyes was the monster - a massive beast with sharp teeth, glowing red eyes, and claws that could crush a human with ease. The creature roared, a thunderous sound that shook the forest to its core.\n\nFear surged through Anya, but she refused to let it consume her. She drew the sword from its sheath and charged towards the monster. The blade shimmered in the sunlight, and as it struck the beast's hide, a blinding light erupted, enveloping everything in its radiance.\n\nWhen the light faded, the monster was gone, and in its place was a pile of shattered crystals. Anya had defeated the creature with the magic of her backpack, proving that even the smallest of objects could hold the greatest of powers.\n\nAs she and Samuel returned to the town, they were greeted as heroes. The people of Willow Creek rejoiced, and the legend of Anya, the girl with the magic backpack, was passed down through generations. And so, Anya continued her adventures, using the backpack's wonders to make the world a better place, one magical step at a time."""

Markdown(story)

In the quaint town of Willow Creek, nestled amidst rolling hills and whispering willows, resided a young girl named Anya. As she stepped out of the creaky wooden door of her modest cottage, her heart skipped a beat with excitement and anticipation. Today was her first day of school, and she couldn't wait to show off her prized possession - a magical backpack.

Handed down to her from her grandmother, the backpack was no ordinary satchel. Its soft, emerald-green fabric shimmered with an ethereal glow, and its leather straps held secrets that only Anya knew. Within its capacious interior lay an enchanted world, filled with wonders that would ignite her imagination and change her life forever.

Anya's parents, kind-hearted Elise and wise-bearded Edward, bid her farewell with warm embraces. "Remember, my dear," whispered her mother, "use your magic wisely and for good." Her father added, "Always seek knowledge, and let the backpack be your trusted companion."

With a skip in her step, Anya set off towards the town's only schoolhouse. On her way, she passed her best friend, Samuel, a curious and adventurous boy with a mischievous grin. "Hey, Anya," he called out. "Can I see your backpack?"

Anya hesitated for a moment before unzipping the flap and revealing its contents. Samuel's eyes widened in amazement as he peered inside. There, nestled amidst pencils and notebooks, were a shimmering sword, a book of ancient spells, a tiny compass that always pointed north, and a magical key that could open any lock.

Together, they marveled at the backpack's wonders, promising to keep its secrets safe. As they approached the schoolhouse, Anya noticed a group of older children huddled together, their faces etched with fear. Curiosity getting the better of her, she cautiously approached.

"What's wrong?" she asked.

A tall, lanky boy stepped forward. "There's a monster in the forest," he stammered. "It's been terrorizing the town, attacking animals and even people."

Anya's heart sank. The town of Willow Creek was small and peaceful, and the thought of a monster brought a shiver down her spine. She knew she had to do something to protect her family and friends.

Without a moment's hesitation, Anya opened her backpack and retrieved the shimmering sword. With a determined gleam in her eye, she turned to her terrified peers. "Don't worry," she said, her voice steady. "I'll take care of it."

With Samuel close behind her, Anya ventured into the shadowy depths of the forest. The trees seemed to whisper secrets as she passed, and the undergrowth rustled with unseen creatures. As they walked deeper into the forest, the air grew heavy and the ground beneath their feet trembled.

Suddenly, they came to a clearing, and there before their eyes was the monster - a massive beast with sharp teeth, glowing red eyes, and claws that could crush a human with ease. The creature roared, a thunderous sound that shook the forest to its core.

Fear surged through Anya, but she refused to let it consume her. She drew the sword from its sheath and charged towards the monster. The blade shimmered in the sunlight, and as it struck the beast's hide, a blinding light erupted, enveloping everything in its radiance.

When the light faded, the monster was gone, and in its place was a pile of shattered crystals. Anya had defeated the creature with the magic of her backpack, proving that even the smallest of objects could hold the greatest of powers.

As she and Samuel returned to the town, they were greeted as heroes. The people of Willow Creek rejoiced, and the legend of Anya, the girl with the magic backpack, was passed down through generations. And so, Anya continued her adventures, using the backpack's wonders to make the world a better place, one magical step at a time.

The code used to generate the short story is shown below. Feel free to generate your own short story.

```python
from google.api_core import retry


model = genai.GenerativeModel(
    model_name='models/gemini-1.5-pro'
)

response = model.generate_content(
    """
    Write a long story about a girl with magic backpack, 
    her family, and at least one other character. 
    Make sure everyone has names. Don't forget to
    describe the contents of the backpack, and where 
    everyone and everything starts and ends up.
    """, 
    request_options={
        'retry': retry.Retry()
    }
)
      
story = response.text
```


The Gemini API provides an endpoint for counting the number of tokens in a request [`GenerativeModel.count_tokens`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#count_tokens). Let's count the number of tokens in this short story:

In [4]:
MODEL_NAME = 'models/gemini-1.5-flash-latest'

model = genai.GenerativeModel(MODEL_NAME)
model.count_tokens(story)

total_tokens: 818

In this example you can see the `gemini-1.5-flash` model has about 1 million tokens context window. If you need more, `gemini-1.5-pro` has an even bigger 2 million tokens context window.

In [5]:
model_info = genai.get_model(MODEL_NAME)

print(
    f'{MODEL_NAME} - input limit: {model_info.input_token_limit}, '
    f'output limit: {model_info.output_token_limit}'
)

models/gemini-1.5-flash-latest - input limit: 1048576, output limit: 8192


## Use function calling to extract structured data

With function calling your function and its parameters are described to the API as a `genai.protos.FunctionDeclaration`. In basic cases the SDK can build the `FunctionDeclaration` from the function and its annotations. The SDK doesn't currently handle the description of nested `OBJECT` (`dict`) parameters. So you'll need to define them explicitly, for now.

### Define the schema

Define a class for each of the entity we are trying to extract from the text. The model uses the class name, docstring, attributes, and attribute type annotations to decide what to extract from the text. Therefore, it is important to be as clear and detailed as possible when defining the classes.

In [6]:
from typing_extensions import TypedDict


class Character(TypedDict):
    """
    Information about a character extracted from the text.
    A character can be a person, animal or creature.

    Attributes:
        name: Name of the character.
        description: Short summary describing the character.
        place_name: Name of the place where the character 
            is first encountered.
    """
    name: str
    description: str
    place_name: str
    

class Place(TypedDict):
    """
    Information about a place extracted from the text.
    A place is where the main character lives in 
    or has visited.

    Attributes:
        name: Name of the place.
        description: Short summary describing the place.
    """
    
    name: str
    description: str


class Thing(TypedDict):
    """
    Information about an inanimate object or 
    thing extracted from the text. The thing
    must belong to at least one character from 
    the text.

    Attributes:
        name: Name of the object or thing.
        description: Short summary describing the thing.
        owner_name: Name of character that owns the thing.
    """
    
    name:str
    description:str
    owner_name:str


class Relationship(TypedDict):
    """
    Information about a relationship between 
    two characters extracted from the text.

    Attributes:
        char_1_name: Name of first character in the relationship.
        char_2_name: Name of second character in the relationship.
        relationship: Short summary describing the relationship 
            between the two characters, 
            e.g., `Alice` is the daughter of `Tom`, 
            `Harold` and `Kumar` are best friends
    """
    
    char_1_name: str
    char_2_name: str
    relationship: str

When you create a function to be used in a function call by the model, you should include as much detail as possible in the function and parameter descriptions. The generative model uses this information to determine which function to select and how to provide values for the parameters in the function call. 
> Take a look at 
[Intro to Function Calling with Gemini API](https://ai.google.dev/gemini-api/docs/function-calling)
to see the best practices for function calling.

Now build the `FunctionDeclaration`:

In [7]:
def add_to_database(
    people: list[Character],
    places: list[Place],
    things: list[Thing],
    relationships: list[Relationship]
) -> bool:
    """
    Add the extracted entities from the text to the database.

    Args:
      people: List of zero or more characters found in the text.
      places: List of zero or more places found in the text.
      things: List of zero or more things found in the text.
      relationships: List of zero or more relationships between 
          two characters found in the text.

    Returns:
        `True` if entities are successfully added;
        `False` otherwise.
    """
    return True

### Call the API

Now you can pass this `FunctionDeclaration` to the `tools` argument of the `genai.GenerativeModel` constructor (the constructor would also accept an equivalent JSON representation of the function declaration).

> See also: [Function calling tutorial](https://ai.google.dev/gemini-api/docs/function-calling/tutorial?lang=python)

In [11]:
model = genai.GenerativeModel(
    model_name=MODEL_NAME,
    tools = [add_to_database]
)

Let's peek inside the model's `_tools` attribute, we can see how it describes the function you just passed to the model:

In [19]:
model._tools.to_proto()

[function_declarations {
   name: "add_to_database"
   description: "\n    Add the extracted entities from the text to the database.\n\n    Args:\n      people: List of zero or more characters found in the text.\n      places: List of zero or more places found in the text.\n      things: List of zero or more things found in the text.\n      relationships: List of zero or more relationships between \n          two characters found in the text.\n\n    Returns:\n        `True` if entities are successfully added;\n        `False` otherwise.\n    "
   parameters {
     type_: OBJECT
     properties {
       key: "people"
       value {
         type_: ARRAY
         items {
           type_: OBJECT
           description: "Information about a character extracted from the text.\nA character can be a person, animal or creature.\n\nAttributes:\n    name: Name of the character.\n    description: Short summary describing the character.\n    place_name: Name of the place where the character \n   

This returns the list of `genai.protos.Tool` objects that would be sent to the API. These are Google protobuf classes. Each `genai.protos.Tool` (1 in this case) contains a list of `genai.protos.FunctionDeclarations`, which describe a function and its arguments.

Let's define the prompt to feed into the model:

In [20]:
prompt = f"""\
Given the following story:

{story}

---

Please add the characters, places, things, and 
relationships from this story to the database:
"""

Each time you call the API the SDK will send the tools along with your prompt, and the model should call that function you defined.

For the temperature parameter, use `0` or another low value. This instructs the model to generate more confident results and reduces hallucinations.

In [21]:
result = model.generate_content(
    prompt,
    generation_config=genai.types.GenerationConfig(
        temperature=0.0
    )
)

Now there is no text to parse. The result _is_ a datastructure.

In [31]:
'text' in result.candidates[0].content.parts[0]

False

In [32]:
'function_call' in result.candidates[0].content.parts[0]

True

In [29]:
function_call = result.candidates[0].content.parts[0].function_call
print(type(function_call))

<class 'google.ai.generativelanguage_v1beta.types.content.FunctionCall'>


The `genai.protos.FunctionCall` class is based on Google Protocol Buffers, convert it to a more familiar JSON compatible object:

In [30]:
import json

print(
    json.dumps(
        type(function_call).to_dict(function_call), 
        indent=2
    )
)

{
  "name": "add_to_database",
  "args": {
    "relationships": [
      {
        "relationship": "Anya is the daughter of Elise",
        "char_1_name": "Anya",
        "char_2_name": "Elise"
      },
      {
        "char_2_name": "Edward",
        "relationship": "Anya is the daughter of Edward",
        "char_1_name": "Anya"
      },
      {
        "char_2_name": "Samuel",
        "relationship": "Anya is best friends with Samuel",
        "char_1_name": "Anya"
      }
    ],
    "places": [
      {
        "name": "Willow Creek",
        "description": "A quaint town nestled amidst rolling hills and whispering willows"
      },
      {
        "description": "A shadowy forest where the monster lives",
        "name": "Forest"
      }
    ],
    "things": [
      {
        "description": "A backpack with magical powers",
        "name": "Magical Backpack",
        "owner_name": "Anya"
      },
      {
        "description": "A shimmering sword",
        "owner_name": "Anya",
     

## Conclusion

While the API can handle structured data extraction problems with pure text input and text output, using function calling is likely more reliable since it lets us define a strict schema, and eliminates a potentially error-prone parsing step.